In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime

import torch
from torch import nn
import torch.nn.functional as F
from skorch import dataset
from skorch import NeuralNetRegressor
from skorch.callbacks import EarlyStopping, LRScheduler, GradientNormClipping

from sklearn.ensemble import BaggingRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.dummy import DummyRegressor
from sklearn.utils.fixes import loguniform

import scipy.stats as stats

from panel_utils import *

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
    
torch.cuda.get_device_name(0)

In [ ]:
df = pd.read_csv("../_data/panelbigcap.csv", index_col="Date")
df = df.loc[df.index <= "2019.12.31"]
scale_interact_sort(df=df, micro_cols=micro_cols, macro_cols=macro_cols, interact=False)
df = df.round(4)

In [ ]:
%store -r df

In [ ]:
train = df.loc[df.index <= "2009.12.31"]
test = df.loc[df.index > "2009.12.31"]

train.shape, test.shape

In [ ]:
class RegressorModule(nn.Module):
    def __init__(self):
        super(RegressorModule, self).__init__()

        self.dense0 = nn.Linear(57, 128)
        self.nonlin0 = torch.tanh
        self.dropout0 = nn.Dropout(0.05)
        self.dense1 = nn.Linear(128, 64)
        self.nonlin1 = torch.tanh
        self.dropout1 = nn.Dropout(0.05)
        self.dense2 = nn.Linear(64, 32)
        self.nonlin2 = torch.tanh
        self.dropout2 = nn.Dropout(0.05)
        self.dense3 = nn.Linear(32, 16)
        self.nonlin3 = torch.tanh
        self.dropout3 = nn.Dropout(0.05)
        self.dense4 = nn.Linear(16, 8)
        self.nonlin4 = torch.tanh
        self.output = nn.Linear(8, 1)

    def forward(self, X):
        X = self.nonlin0(self.dense0(X))
        X = self.dropout0(X)
        X = self.nonlin1(self.dense1(X))
        X = self.dropout1(X)
        X = self.nonlin2(self.dense2(X))
        X = self.dropout2(X)
        X = self.nonlin3(self.dense3(X))
        X = self.dropout3(X)
        X = self.nonlin4(self.dense4(X))
        X = self.output(X)
        return X.squeeze(-1)

In [ ]:

bestparams = []
predictions = []
naive_predictions = []
train_end_dates = []

naive_mean = DummyRegressor(strategy="mean")

for i in np.arange(2009,2019,1):
    train_realtime = df.loc[pd.to_datetime(df.index).year <= i]
    test_realtime = df.loc[(pd.to_datetime(df.index).year > i) & (pd.to_datetime(df.index).year < i+2)]

    X_train, y_train = train_realtime.drop(["EXCESS_RETURN_T+1"], axis=1).values.astype('float32'), train_realtime["EXCESS_RETURN_T+1"].values.astype('float32')
    X_test, y_test = test_realtime.drop(["EXCESS_RETURN_T+1"], axis=1).values.astype('float32'), test_realtime["EXCESS_RETURN_T+1"].values.astype('float32')

    net_regr = NeuralNetRegressor(
            RegressorModule,
            max_epochs=100,
            optimizer=torch.optim.Adam,
            optimizer__lr=0.01,
            optimizer__weight_decay=0.1,
            device=device,
            train_split= dataset.ValidSplit(cv=0.2), 
            criterion = torch.nn.MSELoss,
            batch_size=2048,
            iterator_train__shuffle=True,
            callbacks=[EarlyStopping(patience=5,monitor='valid_loss',threshold=0.001), LRScheduler(policy='WarmRestartLR', monitor='train_loss'),
                       GradientNormClipping(gradient_clip_value=None)],
            verbose=0
    )

    bag = BaggingRegressor(net_regr, n_estimators=10)
        
    bag.fit(X_train, y_train)
    naive_mean.fit(X_train, y_train)

    predictions.extend(bag.predict(X_test).tolist())
    naive_predictions.extend(naive_mean.predict(X_test).tolist())
    train_end_dates.append(pd.to_datetime(train_realtime.index[-1]).strftime('%Y-%m-%d'))

In [ ]:
print("zeroOOSR2", modOOSR2(test["EXCESS_RETURN_T+1"].values, np.array(predictions)))
print("meanOOSR2: ", OOSR2(
    test["EXCESS_RETURN_T+1"].values, 
    np.array(predictions), 
    np.array(naive_predictions)
))